# DB2-Salesforce populate survey URLs obtain from DB2

In [1]:
# Parameters
# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'nanoHUB_user_ID__c'
object_id = 'Contact'

# login parameters to be handled by Papermill
from nanoHUB.application import Application

application = Application.get_instance()
nanohub_db = application.new_db_engine('nanohub')

salesforce = application.new_salesforce_engine()
db_s = salesforce

nanoHUB - Serving Students, Researchers & Instructors


2021-08-26 00:40:48,088| INFO    | 0 keys loaded from agent


2021-08-26 00:40:48,088 - [INFO] sshtunnel.SSHTunnelForwarder [sshtunnel.get_agent_keys:1060]: 0 keys loaded from agent


2021-08-26 00:40:48,091| INFO    | 0 key(s) loaded


2021-08-26 00:40:48,091 - [INFO] sshtunnel.SSHTunnelForwarder [sshtunnel.get_keys:1117]: 0 key(s) loaded


...

2021-08-26 00:40:48,093 - [INFO] sshtunnel.SSHTunnelForwarder [sshtunnel.__init__:978]: Connecting to gateway: db2.nanohub.org:22 as user 'saxenap'


2021-08-26 00:40:48,094 - [INFO] nanoHUB.containers.dataaccess [connection.get_connection_for:101]: Started SSH Tunnel with db2.nanohub.org


2021-08-26 00:40:48,389| INFO    | Opening tunnel: 0.0.0.0:54331 <> 127.0.0.1:3306


2021-08-26 00:40:48,389 - [INFO] sshtunnel.SSHTunnelForwarder [sshtunnel._serve_forever_wrapper:1433]: Opening tunnel: 0.0.0.0:54331 <> 127.0.0.1:3306


Obtained Salesforce access token ...... True


In [2]:
import pandas as pd
import datetime

## Get the URL code from DB2


In [3]:
# page_id indicates the question
url_code_df = pd.read_sql_query("select user_id, code, expiration from jos_reply_access_codes \
where page_id = 2 order by user_id desc;",nanohub_db) #limit 5000

new_pref_code_df = pd.read_sql_query("select user_id, code, expiration from jos_reply_access_codes \
where page_id = 1 order by user_id desc;",nanohub_db) # limit 5000

In [4]:
display(url_code_df.head(2))

,user_id,code,expiration
0,336317,843e37a2e875d13b6cb727b14fb6c99c431708b3129a07...,2021-09-12
1,336315,63b581d90ad85323a5b8478e494ace7561a2dc4cc73051...,2021-09-12


In [5]:
## populating full url codes
url_base = "https://nanohub.org/reply/pages/2?code="

from copy import deepcopy
codes = deepcopy(url_code_df['code'])

#import time
#st = time.time()
codes = codes.apply(lambda x: url_base+x)
#tt = st-time.time()
#print(tt)
print(codes[0])


https://nanohub.org/reply/pages/2?code=843e37a2e875d13b6cb727b14fb6c99c431708b3129a079b8847f8e2fc878aac


In [6]:
## add the new code to the dataframe
url_code_df['code'] = codes
url_code_df = url_code_df.rename(columns={"user_id": "nanoHUB_user_ID__c", "code":"nh_use_survey_link__c"})
display(url_code_df.head(2))

,nanoHUB_user_ID__c,nh_use_survey_link__c,expiration
0,336317,https://nanohub.org/reply/pages/2?code=843e37a...,2021-09-12
1,336315,https://nanohub.org/reply/pages/2?code=63b581d...,2021-09-12


In [7]:
## determining the new url survey links
url_base1 = 'https://nanohub.org/reply/email-subscriptions?code='

codes = deepcopy(new_pref_code_df['code'])

codes = codes.apply(lambda x: url_base1+x)

print(codes[0])
new_pref_code_df['code'] = codes

https://nanohub.org/reply/email-subscriptions?code=79ae5a59f40e190368579cab54b2a0ffae22462387f7df3b29593b11576b4ab5


In [8]:
display(new_pref_code_df.head(5))

,user_id,code,expiration
0,336317,https://nanohub.org/reply/email-subscriptions?...,2021-09-12
1,336315,https://nanohub.org/reply/email-subscriptions?...,2021-09-12
2,336314,https://nanohub.org/reply/email-subscriptions?...,2021-09-12
3,336313,https://nanohub.org/reply/email-subscriptions?...,2021-09-12
4,336312,https://nanohub.org/reply/email-subscriptions?...,2021-09-12


In [9]:
# expiration dates can be different, so do a left join instead of "url_code_df['new_preferences_survey_url__c'] = codes"
unif_df = pd.merge(url_code_df,new_pref_code_df,how='left',right_on='user_id',left_on='nanoHUB_user_ID__c')

display(unif_df.head(2))
display(unif_df.tail(2))

,nanoHUB_user_ID__c,nh_use_survey_link__c,expiration_x,user_id,code,expiration_y
0,336317,https://nanohub.org/reply/pages/2?code=843e37a...,2021-09-12,336317,https://nanohub.org/reply/email-subscriptions?...,2021-09-12
1,336315,https://nanohub.org/reply/pages/2?code=63b581d...,2021-09-12,336315,https://nanohub.org/reply/email-subscriptions?...,2021-09-12


,nanoHUB_user_ID__c,nh_use_survey_link__c,expiration_x,user_id,code,expiration_y
248866,1000,https://nanohub.org/reply/pages/2?code=ejjJycB...,2021-09-12 02:59:59,1000,https://nanohub.org/reply/email-subscriptions?...,2021-08-18 14:31:42
248867,998,https://nanohub.org/reply/pages/2?code=rWB03ln...,2021-09-12 02:59:59,998,https://nanohub.org/reply/email-subscriptions?...,2021-08-18 14:31:42


In [10]:
# some formatting for SF
unif_df = unif_df.rename(columns={'expiration_x':'nh_use_expire__c',\
                'code':'new_preferences_survey_url__c','expiration_y':'new_prefs_expire__c'})
unif_df = unif_df.drop(columns=['user_id']) #unified_dataframe

In [11]:
display(unif_df.head(2))
display(unif_df.tail(2))

,nanoHUB_user_ID__c,nh_use_survey_link__c,nh_use_expire__c,new_preferences_survey_url__c,new_prefs_expire__c
0,336317,https://nanohub.org/reply/pages/2?code=843e37a...,2021-09-12,https://nanohub.org/reply/email-subscriptions?...,2021-09-12
1,336315,https://nanohub.org/reply/pages/2?code=63b581d...,2021-09-12,https://nanohub.org/reply/email-subscriptions?...,2021-09-12


,nanoHUB_user_ID__c,nh_use_survey_link__c,nh_use_expire__c,new_preferences_survey_url__c,new_prefs_expire__c
248866,1000,https://nanohub.org/reply/pages/2?code=ejjJycB...,2021-09-12 02:59:59,https://nanohub.org/reply/email-subscriptions?...,2021-08-18 14:31:42
248867,998,https://nanohub.org/reply/pages/2?code=rWB03ln...,2021-09-12 02:59:59,https://nanohub.org/reply/email-subscriptions?...,2021-08-18 14:31:42


## Send to SF

In [12]:
db_s.object_id = object_id
db_s.external_id = external_id

# send data to Salesforce
db_s.send_data(unif_df)

[Success] Bulk job creation successful. Job ID = 7505w00000ZJL5yAAH
hello


[Success] CSV upload successful. Job ID = 7505w00000ZJL5yAAH


[Success] Closing job successful. Job ID = 7505w00000ZJL5yAAH


In [13]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000ZJL5yAAH',
 'operation': 'upsert',
 'object': 'Contact',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-08-26T04:41:08.000+0000',
 'systemModstamp': '2021-08-26T04:42:43.000+0000',
 'state': 'UploadComplete',
 'externalIdFieldName': 'nanoHUB_user_ID__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [14]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_failed_results())

''
